In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        # TODO: Implement!
        vFuncChange = 0
        for s in range(env.nS):
            value = 0
            for a in range(env.nA):
                probOfa = policy[s][a]
                for prob, next_state, reward, done in env.P[s][a]:
                    value += probOfa * prob * (reward + discount_factor * V[next_state])
            vFuncChange = max(vFuncChange, abs(V[s] - value))
            V[s] = value
        if vFuncChange < theta:
            break
    return np.array(V)

In [4]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        #TODO: Implement!
        V = policy_eval_fn(policy, env, discount_factor)
        optimal_policy = True
        
        for s in range(env.nS):
            v = V[s]
            max_q_value = float("-inf")
            optimal_a = -1
            for a in range(env.nA):
                q_value = 0
                for prob, next_state, reward, done in env.P[s][a]:
                    q_value += prob * (reward + discount_factor*V[next_state])
                if max_q_value < q_value:
                    optimal_a = a
                    max_q_value = q_value
            
            policy_a = np.argmax(policy[s])

            if optimal_a != policy_a:
                optimal_policy = False
            policy[s] = np.eye(env.nA)[optimal_a]
                
        if optimal_policy:        
            return policy, V

In [8]:
policy, v = policy_improvement(env)
v = policy_eval(policy, env, 1, 0.000000000001)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [0.   0.   1.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [1.   0.   0.   0.  ]
 [0.   0.   1.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   0.   1.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [1 1 1 2]
 [1 1 1 0]]

Value Function:
[ 0.         -1.         -2.         -3.         -3.90909091 -3.72727273
 -3.         -2.         -4.         -3.         -2.         -1.
 -3.         -2.         -1.          0.        ]

Reshaped Grid Value Function:
[[ 0.         -1.         -2.         -3.        ]
 [-3.90909091 -3.72727273 -3.         -2.        ]
 [-4.         -3.         -2.         -1.        ]
 [-3.         -2.         -1.          0.        ]]



In [9]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

AssertionError: 
Arrays are not almost equal to 2 decimals

Mismatched elements: 3 / 16 (18.8%)
Max absolute difference: 2.90909091
Max relative difference: 2.90909091
 x: array([ 0.  , -1.  , -2.  , -3.  , -3.91, -3.73, -3.  , -2.  , -4.  ,
       -3.  , -2.  , -1.  , -3.  , -2.  , -1.  ,  0.  ])
 y: array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])

In [13]:
np.eye(5)[3]

array([0., 0., 0., 1., 0.])